In [230]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
import plotly.express as px
project_path = 'D:\esperimenti\programming_database\houses_price'
data = pd.read_csv(project_path+'\data\\train.csv')
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [247]:
nan_features = data.isnull().sum().where(data.isnull().sum() != 0 ).dropna()
sale_corr = data.corr()['SalePrice'].sort_values()

#get only features where corr > 0.5 and interesting features for a possible buyer
top_sale_corr = sale_corr.sort_values().where(sale_corr > 0.45).dropna()
top_sale_corr_indexes = top_sale_corr.index
top_sale_corr_indexes = top_sale_corr_indexes.append(pd.Index(["MSZoning"]))

#Create new dataset and check that NaN values are not present (Drop Column 'GarageYrBlt' and delete rows where 'MasVnrArea' equals to Nan  )
new_data = data[top_sale_corr_indexes]
new_data = new_data.drop("GarageYrBlt", axis=1)
new_data = new_data.drop("MasVnrArea",axis=1)
new_data = new_data.drop("YearRemodAdd",axis=1)
new_data
#new_data.dropna(subset = ["MasVnrArea"], inplace=True)

,Fireplaces,YearBuilt,TotRmsAbvGrd,FullBath,1stFlrSF,TotalBsmtSF,GarageArea,GarageCars,GrLivArea,OverallQual,SalePrice,MSZoning
0,0,2003,8,2,856,856,548,2,1710,7,208500,RL
1,1,1976,6,2,1262,1262,460,2,1262,6,181500,RL
2,1,2001,6,2,920,920,608,2,1786,7,223500,RL
3,1,1915,7,1,961,756,642,3,1717,7,140000,RL
4,1,2000,9,2,1145,1145,836,3,2198,8,250000,RL
...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1,1999,7,2,953,953,460,2,1647,6,175000,RL
1456,2,1978,7,2,2073,1542,500,2,2073,6,210000,RL
1457,2,1941,9,2,1188,1152,252,1,2340,7,266500,RL
1458,0,1950,5,1,1078,1078,240,1,1078,5,142125,RL


In [133]:
#Graph number of house per year and average price
houses_per_year = new_data.groupby("YearBuilt").count()["SalePrice"]
houses_per_year.rename("Total houses built", inplace=True)
median_sale_price = new_data[["YearBuilt","SalePrice"]].groupby("YearBuilt").median('SalePrice').sort_values(by="YearBuilt")

fig = px.line(
    y=median_sale_price["SalePrice"].div(10000).tolist(), 
    x=median_sale_price.index,
    color=px.Constant("Median Price (x 10k)"),
    labels=dict(x="Year", y="Median Price (x 10k)", color="Time Period"),
    title="")
fig.add_bar(
    y=houses_per_year.values.tolist(), 
    x=median_sale_price.index,
    name="Nr. houses")

fig.show()

In [227]:
#Number house per year and tipology of house

houses_per_year_type = new_data.groupby(["YearBuilt","MSZoning"]).count()
houses_per_year_type.reset_index(level=1, inplace=True)
houses_per_year_type.reset_index(level=0, inplace=True)
houses_per_year_type = houses_per_year_type[["YearBuilt","MSZoning","SalePrice"]].rename(columns={'SalePrice':"count"})
houses_per_year_type['MSZoning'] = houses_per_year_type['MSZoning'].replace(['A'],'Agriculture')
houses_per_year_type['MSZoning'] = houses_per_year_type['MSZoning'].replace(['C (all)'],'Commercial')
houses_per_year_type['MSZoning'] = houses_per_year_type['MSZoning'].replace(['FV'],'Floating Village Residential')
houses_per_year_type['MSZoning'] = houses_per_year_type['MSZoning'].replace(['I'],'Industrial')
houses_per_year_type['MSZoning'] = houses_per_year_type['MSZoning'].replace(['RH'],'Resid. High Density')
houses_per_year_type['MSZoning'] = houses_per_year_type['MSZoning'].replace(['RL'],'Resid. Low Density')
houses_per_year_type['MSZoning'] = houses_per_year_type['MSZoning'].replace(['RP'],'Resid. Low Density Park')
houses_per_year_type['MSZoning'] = houses_per_year_type['MSZoning'].replace(['RM'],'Resid. Medium Density')

fig2 = px.bar(houses_per_year_type, 
    x="YearBuilt",
    y="count", 
    color="MSZoning", 
    title="Numero di case per zona ",
    labels={'count':'Numero Case','YearBuilt':'Anno construzione'})
fig2.show()


In [255]:
#Paramenters choice based an price correlazion
corr_map = new_data.corr().sort_values("SalePrice")["SalePrice"].reset_index(level=0).rename(columns={"index":"Feature"})

fig3 = px.bar(corr_map, x='SalePrice', y='pop',
             hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
             labels={'pop':'population of Canada'}, height=400)
fig3.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['SalePrice'] but received: year

In [241]:


fig4 = make_subplots(rows=1, cols=2)
fig4.add_trace(
    go.Scatter(x=new_data["Fireplaces"], y=new_data["SalePrice"],mode='markers'),
    row=1, col=1
)


# fig3 = px.scatter(new_data[["Fireplaces","SalePrice"]]
#     , x='Fireplaces', y='SalePrice')
fig4.show()
corr_map

Fireplaces      0.466929
YearBuilt       0.522897
TotRmsAbvGrd    0.533723
FullBath        0.560664
1stFlrSF        0.605852
TotalBsmtSF     0.613581
GarageArea      0.623431
GarageCars      0.640409
GrLivArea       0.708624
OverallQual     0.790982
Name: SalePrice, dtype: float64

In [265]:
px.data.medals_wide(indexed=True)
el = new_data.corr().sort_values("SalePrice")["SalePrice"]
#px.data.gapminder().query("country == 'Canada'")
el.reset_index(level=0).rename(columns={"index":"Feature"})

,Feature,SalePrice
0,Fireplaces,0.466929
1,YearBuilt,0.522897
2,TotRmsAbvGrd,0.533723
3,FullBath,0.560664
4,1stFlrSF,0.605852
5,TotalBsmtSF,0.613581
6,GarageArea,0.623431
7,GarageCars,0.640409
8,GrLivArea,0.708624
9,OverallQual,0.790982
